In [ ]:
!pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3ee21fba2c20988da22827866608631a380af456ffa9e2d3eb7379946a7336da
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.6 MB/s eta 0:00:00
Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 31.8 MB/s eta 0:00:00
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [ ]:
from datasets import load_dataset

In [ ]:
billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)
billsum_train = billsum["train"].train_test_split(test_size=0.2)  # Further split for training and validation
billsum_test = billsum["test"]

In [ ]:
from transformers import LEDTokenizer
checkpoint="allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(checkpoint)
print(tokenizer.max_len_single_sentence)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

16382


In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=4000, truncation=True)

    labels = tokenizer(text_target=examples["summary"],max_length=200, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_billsum_train = billsum_train.map(preprocess_function, batched=True)
tokenized_billsum_test = billsum_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate
rouge = evaluate.load("rouge")

In [ ]:

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args =  Seq2SeqTrainingArguments(
    output_dir="LEDbillsumtest1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_steps=100,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum_train["train"],
    eval_dataset=tokenized_billsum_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Input ids are automatically padded from 2302 to 3072 to be a multiple of `config.attention_window`: 1024


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.902200,1.731199,0.138800,0.073100,0.121900,0.126300,20.000000
2,1.590800,1.636442,0.150000,0.086900,0.132700,0.139200,20.000000
3,1.384200,1.586565,0.146400,0.085200,0.131400,0.135800,20.000000
4,1.241300,1.575768,0.144200,0.081200,0.128900,0.132800,20.000000
5,1.087200,1.585021,0.146800,0.085900,0.131200,0.136400,20.000000
6,1.058500,1.582567,0.136400,0.073900,0.121400,0.125800,20.000000
7,0.946000,1.611023,0.142700,0.079400,0.126500,0.131600,20.000000
8,0.899800,1.608853,0.143500,0.081100,0.127500,0.132000,20.000000
9,0.854700,1.617352,0.142100,0.079000,0.126100,0.131000,20.000000
10,0.839400,1.626070,0.144400,0.081800,0.128000,0.133200,20.000000


Input ids are automatically padded from 1376 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3665 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2389 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1520 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3860 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2866 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2506 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2657 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1500 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3782 to 4096 to

TrainOutput(global_step=2640, training_loss=1.1911002505909314, metrics={'train_runtime': 2023.4089, 'train_samples_per_second': 3.909, 'train_steps_per_second': 1.305, 'total_flos': 1.425333380700672e+16, 'train_loss': 1.1911002505909314, 'epoch': 10.0})

In [ ]:
evaluation_results = trainer.evaluate(tokenized_billsum_test)

Input ids are automatically padded from 3346 to 4096 to be a multiple of `config.attention_window`: 1024


Input ids are automatically padded from 3558 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3540 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1927 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1998 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2350 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2528 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2659 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3838 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2756 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1551 to 2048 to

In [ ]:
print("Evaluation Results:", evaluation_results)

Evaluation Results: {'eval_loss': 1.6576324701309204, 'eval_rouge1': 0.14, 'eval_rouge2': 0.0761, 'eval_rougeL': 0.1222, 'eval_rougeLsum': 0.1279, 'eval_gen_len': 20.0, 'eval_runtime': 54.8769, 'eval_samples_per_second': 4.519, 'eval_steps_per_second': 1.512, 'epoch': 10.0}


In [ ]:
evaluation_results

{'eval_loss': 1.6576324701309204,
 'eval_rouge1': 0.14,
 'eval_rouge2': 0.0761,
 'eval_rougeL': 0.1222,
 'eval_rougeLsum': 0.1279,
 'eval_gen_len': 20.0,
 'eval_runtime': 54.8769,
 'eval_samples_per_second': 4.519,
 'eval_steps_per_second': 1.512,
 'epoch': 10.0}